In [31]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import math

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [32]:
#Read From File
dataSet = unpickle("data_batch_1")
labels=dataSet[b'labels']
data=dataSet[b'data']
filenames=dataSet[b'filenames']
MaxSize=1000 #10000 max
labels=labels[:MaxSize]
data=data[:MaxSize]
filenames=filenames[:MaxSize]

data=data/255

#Train/Test Split
X_all, X_test, y_all, y_test = train_test_split(data,labels,test_size=0.2,random_state=0)
X_train, X_validate, y_train, y_validate = train_test_split(X_all,y_all,test_size=0.2,random_state=0)

In [33]:
middle_layer=100
W1 = np.random.rand(middle_layer,3072)
W2 = np.random.rand(10,middle_layer)
W1 = W1*1e-6
W2 = W2*1e-6
W1 = np.asmatrix(W1)
W2 = np.asmatrix(W2)
B1=np.asmatrix(np.zeros(100)).T

In [34]:
#Additional Functions
def f(W,x):
    row=np.asmatrix(x)
    ans=W*row.T
    return ans

def MtrxPrint(M):
    n,m = M.shape
    print(n,m)
    for i in range(n):
        for j in range(10):
            print(M[i,j],' ',end='')
        print()

In [35]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [36]:
def forwardPredict(W2,W1,b1,x):
    #input layer
    s=np.maximum(0,f(W1,x)+b1)
    #hidden layer
    ans=np.asarray(softmax(f(W2,s.T)))
    predict=np.argmax(ans)
    return predict

In [37]:
def SoftMaxLoss(x,y):
    return -np.log(softmax(x)[y])

def GradientNumericVector(x,y):
    eps=1e-5
    grad=np.zeros(10)
    lsS=SoftMaxLoss(x,y)
    for i in range(10):
        xt=x.copy()
        xt[i]+=eps
        ls=SoftMaxLoss(xt,y)
        grad[i]=(ls-lsS)/eps
    return grad

In [48]:
def backPropLossI(W2,W1,b1,x,y,DW2,DW1,Db1):
#forward
    #input layer
    inp=f(W1,x)+b1
    s=np.maximum(inp,0)
    
    #hidden layer
    hd=f(W2,s.T)
    
    #softmax
    ex=np.exp(hd)
    
    #print(ex)
    sm=np.sum(ex)
    if(sm<1e-5):
        sm=1e-5
    LossI=-np.log(ex[y]/sm)
    
#backward
    #DLossI=1
    Dsm=1/sm
    
    Dhd=ex*Dsm
    Dhd[y]-=1

    Ds=Dhd.T*W2
    
    DhdT=np.asarray(Dhd.T).flatten()
    n=np.shape(W2)[0]
    for i in range(n):
        DW2[i]=s.T*float(DhdT[i])
    
    Dinp=np.asarray(Ds.copy())
    Dinp=Dinp.flatten()
    for i in range(np.shape(Dinp)[0]):
        if inp[i]<=0:
            Dinp[i]=0
            
    Db1=Dinp
    n=np.shape(W1)[0]
    for i in range(n):
        DW1[i]=x*float(Dinp.T[i])
    return LossI

In [39]:
def gradientStep(X,Y,W2,W1,B1,learning_rate):
    DW1_g = np.zeros(middle_layer*3072)
    DW1_g=np.resize(DW1_g,[middle_layer,3072])
    DW2_g = np.zeros(10*middle_layer)
    DW2_g=np.resize(DW2_g,[10,middle_layer])
    Db1_g=np.asmatrix(np.zeros(100)).T
    
    N=len(Y)
    Loss=0
    for i in range(N):
        #init new gradients matrix
        DW1 = np.zeros(middle_layer*3072)
        DW1=np.resize(DW1,[middle_layer,3072])
        DW2 = np.zeros(10*middle_layer)
        DW2=np.resize(DW2_g,[10,middle_layer])
        Db1=np.asmatrix(np.zeros(100)).T
        
        Loss+=backPropLossI(W2,W1,B1,X[i],Y[i],DW2,DW1,Db1)
        DW1_g+=DW1
        DW2_g+=DW2
        Db1_g+=Db1
    
    
    W1-=learning_rate*DW1_g/N
    W2-=learning_rate*DW2_g/N
    B1-=learning_rate*Db1_g/N
    #B2=0 (analitic)
    return Loss/N

In [40]:
def postProcess(W2,W1,B1):
    n,m = np.shape(W2)
    
    for i in range(n):
        for j in range(m):
            if W2.item(i,j)>10:
                W2.itemset((i,j),10)
                
    n,m = np.shape(W1)
    for i in range(n):
        for j in range(m):
            if W1.item(i,j)>10:
                W1.itemset((i,j),10)
                
    n = np.shape(B1)[0]
    for i in range(n):
        if B1.item(i)>10:
            B1.itemset(i,10)

In [41]:
DW1 = np.zeros(middle_layer*3072)
DW1=np.resize(DW1,[middle_layer,3072])
DW2 = np.zeros(10*middle_layer)
DW2=np.resize(DW2,[10,middle_layer])
Db1=np.asmatrix(np.zeros(100)).T

In [42]:
#print(GradientNumericVector(x,0))
#print(SoftMaxLoss(x,0))
backPropLossI(W2,W1,B1,X_test[0],0,DW2,DW1,Db1)
#MtrxPrint(b1)

matrix([[ 2.3025851]])

In [63]:
#learning_rate=1e-3
#for i in range(200):
#    print(gradientStep(X_test,y_test,W2,W1,B1,learning_rate))
#    postProcess(W2,W1,B1)

eps=1e-5
learning_rate=1e-2
while learning_rate > 1e-5:
    before=gradientStep(X_test,y_test,W2,W1,B1,learning_rate)
    print("rate =",learning_rate,"Loss =",before)
    while True:
        after=gradientStep(X_test,y_test,W2,W1,B1,learning_rate)
        change=before-after
        before=after
        print("Loss=",after," change=",change)
        if change < 0:
            break;
        if change < 1e-5:
            break;        
    learning_rate/=10

rate = 0.01 Loss = [[ 1.54278464]]
Loss= [[ 1.54221499]]  change= [[ 0.00056965]]
Loss= [[ 1.54166906]]  change= [[ 0.00054593]]
Loss= [[ 1.54109749]]  change= [[ 0.00057157]]
Loss= [[ 1.54053856]]  change= [[ 0.00055893]]
Loss= [[ 1.53996506]]  change= [[ 0.00057351]]
Loss= [[ 1.53942036]]  change= [[ 0.0005447]]
Loss= [[ 1.53885385]]  change= [[ 0.00056651]]
Loss= [[ 1.53829452]]  change= [[ 0.00055933]]
Loss= [[ 1.53773207]]  change= [[ 0.00056245]]
Loss= [[ 1.53718853]]  change= [[ 0.00054354]]
Loss= [[ 1.53661421]]  change= [[ 0.00057432]]
Loss= [[ 1.53605643]]  change= [[ 0.00055778]]
Loss= [[ 1.53549716]]  change= [[ 0.00055928]]
Loss= [[ 1.53494433]]  change= [[ 0.00055283]]
Loss= [[ 1.53439684]]  change= [[ 0.00054749]]
Loss= [[ 1.5338361]]  change= [[ 0.00056073]]
Loss= [[ 1.53328237]]  change= [[ 0.00055373]]
Loss= [[ 1.5327188]]  change= [[ 0.00056357]]
Loss= [[ 1.53216925]]  change= [[ 0.00054956]]
Loss= [[ 1.53163189]]  change= [[ 0.00053736]]
Loss= [[ 1.53105977]]  chang

KeyboardInterrupt: 

In [62]:
good=0
for i in range(len(X_test)):
    ans=forwardPredict(W2,W1,B1,X_test[i])
    if(ans == y_test[i]):
        good+=1
print("%10.5f"% (good/len(X_test)))

   0.44500


In [50]:
print(np.max(W2))
print(np.max(W1))
print(np.max(B1))

0.0929149729373
0.0122083352346
0.0


In [38]:
data[0]

array([ 59,  43,  50, ..., 140,  84,  72], dtype=uint8)